In [2]:
import pandas as pd
import numpy as np

import geopandas as gpd
from shapely.geometry import Point
import rtree
import pickle
import geohash

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [12]:
mta_df = pd.read_csv("data/mta_turnstile/turnstile_170107.txt")
mta_df = mta_df.append(pd.read_csv("data/mta_turnstile/turnstile_170114.txt"))
mta_df = mta_df.append(pd.read_csv("data/mta_turnstile/turnstile_170121.txt"))
mta_df = mta_df.append(pd.read_csv("data/mta_turnstile/turnstile_170128.txt"))

In [13]:
mta_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,03:00:00,REGULAR,5991546,2028378
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,07:00:00,REGULAR,5991565,2028389
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,11:00:00,REGULAR,5991644,2028441
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,15:00:00,REGULAR,5991971,2028502
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,19:00:00,REGULAR,5992418,2028543


In [14]:
mta_df['DATE'] = pd.to_datetime(mta_df.DATE)

In [15]:
mta_df['month'] = mta_df.DATE.dt.month

In [16]:
mta_df['year'] = mta_df.DATE.dt.year

In [17]:
mta_df['day_of_week'] = mta_df.DATE.dt.weekday_name

In [18]:
mta_df.day_of_week.value_counts()

Tuesday      111687
Wednesday    111432
Friday       110914
Thursday     110841
Monday       110819
Sunday       110728
Saturday     110682
Name: day_of_week, dtype: int64

In [21]:
mta_df[mta_df.STATION == 'CENTRAL AV']

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,month,year,day_of_week
38893,K017,R401,00-00-00,CENTRAL AV,M,BMT,2016-12-31,00:00:00,REGULAR,507074,1691710,12,2016,Saturday
38894,K017,R401,00-00-00,CENTRAL AV,M,BMT,2016-12-31,04:00:00,REGULAR,507081,1691803,12,2016,Saturday
38895,K017,R401,00-00-00,CENTRAL AV,M,BMT,2016-12-31,08:00:00,REGULAR,507099,1691827,12,2016,Saturday
38896,K017,R401,00-00-00,CENTRAL AV,M,BMT,2016-12-31,12:00:00,REGULAR,507175,1691884,12,2016,Saturday
38897,K017,R401,00-00-00,CENTRAL AV,M,BMT,2016-12-31,16:00:00,REGULAR,507256,1692008,12,2016,Saturday
38898,K017,R401,00-00-00,CENTRAL AV,M,BMT,2016-12-31,20:00:00,REGULAR,507313,1692225,12,2016,Saturday
38899,K017,R401,00-00-00,CENTRAL AV,M,BMT,2017-01-01,00:00:00,REGULAR,507371,1692400,1,2017,Sunday
38900,K017,R401,00-00-00,CENTRAL AV,M,BMT,2017-01-01,04:00:00,REGULAR,507412,1692476,1,2017,Sunday
38901,K017,R401,00-00-00,CENTRAL AV,M,BMT,2017-01-01,08:00:00,REGULAR,507441,1692499,1,2017,Sunday
38902,K017,R401,00-00-00,CENTRAL AV,M,BMT,2017-01-01,12:00:00,REGULAR,507472,1692535,1,2017,Sunday
